# Convolutional networks with Nengo DL and Nengo Loihi

In this example we will show how to build and train a convolutional network in NengoDL, and then deploy that network on Loihi.

We'll use the basic MNIST dataset to demonstrate the steps. The input data are images of handwritten digits, and the goal is for the network to classify each image as 0-9.

We will assume here that the reader is somewhat familiar with NengoDL, and focus on the issue of how to use NengoDL to train a network for Loihi.  For a more basic introduction to NengoDL, check out [the documentation](https://www.nengo.ai/nengo-dl) and [examples](https://www.nengo.ai/nengo-dl/examples).

Note: this notebook requires that you do a [developer installation of NengoDL](https://www.nengo.ai/nengo-dl/installation.html#developer-installation) and switch to the `conv_transform` branch (`git checkout origin/conv_transform`).  This is just temporary, as the Loihi convolution implementation is currently a work in progress.  Soon it will be fully supported in the Nengo ecosystem.

In [ ]:
%matplotlib inline

import gzip
import os
import pickle
from urllib.request import urlretrieve

import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import nengo
import nengo_dl

import nengo_loihi

In [ ]:
# load mnist dataset
if not os.path.exists('mnist.pkl.gz'):
    urlretrieve('http://deeplearning.net/data/mnist/mnist.pkl.gz',
                'mnist.pkl.gz')

with gzip.open('mnist.pkl.gz') as f:
    train_data, _, test_data = pickle.load(f, encoding="latin1")
train_data = list(train_data)
test_data = list(test_data)
for data in (train_data, test_data):
    one_hot = np.zeros((data[0].shape[0], 10))
    one_hot[np.arange(data[0].shape[0]), data[1]] = 1
    data[1] = one_hot

# plot some examples
for i in range(3):
    plt.figure()
    plt.imshow(np.reshape(train_data[0][i], (28, 28)))
    plt.axis('off')
    plt.title(str(np.argmax(train_data[1][i])));

We'll begin by defining a simple function to build a "convolutional layer".  This is just a `nengo.Connection` and `nengo.Ensemble` put together, but we'll be doing this a lot so we'll use this function to put them together in an easy-to-use bundle.

In [ ]:
def conv_layer(x, *args, activation=True, **kwargs):
    # create a Conv2D transform with the given arguments
    conv = nengo_loihi.Conv2D(*args, **kwargs)
    
    if activation:
        # add an ensemble to implement the activation function
        layer = nengo.Ensemble(conv.output_shape.size, 1).neurons
    else:
        # no nonlinearity, so we just use a node
        layer = nengo.Node(size_in=conv.output_shape.size)
        
    # connect up the input object to the new layer
    nengo.Connection(x, layer, transform=conv)

    # print out the shape information for our new layer
    print("LAYER")
    print(conv.input_shape.shape(), "->", conv.output_shape.shape())

    return layer, conv

Next we define the structure of our network.  Because we need to keep the number of neurons and axons per core below the Loihi hardware limits, we adopt a somewhat unusual network architecture.  We'll have a relatively small core network, so that each layer fits on one Loihi core, and then repeat that network several times in parallel, summing their output.  We can think of this as a variation on [ensemble learning](https://en.wikipedia.org/wiki/Ensemble_averaging_(machine_learning)).

In [ ]:
dt = 0.001  # simulation timestep
presentation_time = 0.1  # input presentation time
max_rate = 100  # neuron firing rates
amp = 1 / max_rate  # neuron spike amplitude (scaled so that the overall output is ~1)
input_shape = nengo_loihi.conv.ImageShape(28, 28, 1, channels_last=False)  # input image shape
n_parallel = 2  # number of parallel network repetitions

with nengo.Network(seed=0) as net:
    # set up the default parameters for ensembles/connections
    nengo_loihi.add_params(net)
    net.config[nengo.Ensemble].neuron_type = nengo.SpikingRectifiedLinear(amplitude=amp)
    net.config[nengo.Ensemble].max_rates = nengo.dists.Choice([max_rate])
    net.config[nengo.Ensemble].intercepts = nengo.dists.Choice([0])
    net.config[nengo.Connection].synapse = None

    # the input node that will be used to feed in input images
    inp = nengo.Node(
        nengo.processes.PresentInput(test_data[0], presentation_time),
        size_out=28 * 28)
    
    # the output node provides the 10-dimensional classification
    out = nengo.Node(size_in=10)
    
    # build parallel copies of the network
    for _ in range(n_parallel):     
        layer, conv = conv_layer(inp, 1, input_shape, kernel_size=1, kernel=np.ones((1, 1, 1, 1)))
        net.config[layer.ensemble].on_chip = False  # first layer is off-chip to translate the images into spikes
        layer, conv = conv_layer(layer, 6, conv.output_shape, strides=2)
        layer, conv = conv_layer(layer, 24, conv.output_shape, strides=2)
        nengo.Connection(layer, out, transform=nengo_dl.dists.Glorot())

    out_p = nengo.Probe(out)
    out_p_filt = nengo.Probe(out, synapse=nengo.Alpha(0.01))

The next step is to optimize the parameters of the network using NengoDL.

First we set up the input/target data for the training and test datasets.

In [ ]:
# set up training data
minibatch_size = 200
train_inputs = {inp: train_data[0][:, None, :]}
train_targets = {out_p: train_data[1][:, None, :]}

# for the test data evaluation we'll be running the network over time
# using spiking neurons, so we need to repeat the input/target data
# for a number of timesteps (based on the presentation_time)
test_inputs = {inp: np.tile(test_data[0][:, None, :],
                            (1, int(presentation_time / dt), 1))}
test_targets = {out_p_filt: np.tile(test_data[1][:, None, :],
                                    (1, int(presentation_time / dt), 1))}

Next we need to define our error functions.  We'll use two different error functions: classification error (the % of images classified incorrectly) as an intuitive measure of how well the network is doing, and crossentropy as the error measure the training process will seek to optimize.

In [ ]:
def crossentropy(outputs, targets):
    return tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(
        logits=outputs, labels=targets))


def classification_error(outputs, targets):
    return 100 * tf.reduce_mean(
        tf.cast(tf.not_equal(tf.argmax(outputs[:, -1], axis=-1),
                             tf.argmax(targets[:, -1], axis=-1)),
                tf.float32))

Now we create the NengoDL simulator and run the training using the `sim.train` function.

More details on how to use NengoDL to optimize a model can be found here https://www.nengo.ai/nengo-dl/training.html.

To speed up this example we can set `do_training=False` to load some pre-trained parameters.  Note, in order to run `do_training=True` you will need to have TensorFlow installed with GPU support.

In [ ]:
do_training = False
with nengo_dl.Simulator(net, minibatch_size=minibatch_size, seed=0) as sim:
    if do_training:
        print("error before training: %.2f%%" %
              sim.loss(test_inputs, test_targets, classification_error))

        # run training
        sim.train(train_inputs, train_targets,
                  tf.train.RMSPropOptimizer(learning_rate=0.001),
                  objective=crossentropy, n_epochs=5)

        print("error after training: %.2f%%" %
              sim.loss(test_inputs, test_targets, classification_error))
        
        sim.save_params("./mnist_params")
    else:
        sim.load_params("./mnist_params")

    # store trained parameters back into the network
    sim.freeze_params(net)

As we built it, the network has no synaptic filters on the neural connections.  This works well during training, but we can see that the error is still somewhat high when we evaluate it using spiking neurons.  We can improve performance by adding synaptic filters to our trained network.

In [ ]:
for conn in net.all_connections:
    conn.synapse = 0.005

if do_training:
    with nengo_dl.Simulator(net, minibatch_size=minibatch_size) as sim:
        print("error w/ synapse: %.2f%%" %
              sim.loss(test_inputs, test_targets, classification_error))

Now we can load our trained network, with synaptic filters, onto Loihi.  This is as easy as passing the network to `nengo_loihi.Simulator` and running it, there is no extra work required.  We will give the network 100 test images, and use that to evaluate the classification error.

In [ ]:
n_presentations = 100
with nengo_loihi.Simulator(net, dt=dt, precompute=True) as sim:
    # run the simulation on Loihi
    sim.run(n_presentations * presentation_time)

    # check classification error
    step = int(presentation_time / dt)
    output = sim.data[out_p_filt][step - 1::step]
    correct = 100 * np.mean(np.argmax(output, axis=-1) !=
                            np.argmax(test_data[1][:n_presentations], axis=-1))
    print("loihi error: %.2f%%" % correct)

We can also plot the output activity from the Loihi network as we show it different test images, to see what this performance looks like in practice.

In [ ]:
n_plots = 10
plt.figure()

plt.subplot(2, 1, 1)
images = test_data[0].reshape(-1, 28, 28, 1)
ni, nj, nc = images[0].shape
allimage = np.zeros((ni, nj * n_plots, nc), dtype=images.dtype)
for i, image in enumerate(images[:n_plots]):
    allimage[:, i * nj:(i + 1) * nj] = image
if allimage.shape[-1] == 1:
    allimage = allimage[:, :, 0]
plt.imshow(allimage, aspect='auto', interpolation='none', cmap='gray')

plt.subplot(2, 1, 2)
plt.plot(sim.trange()[:n_plots * step], sim.data[out_p_filt][:n_plots * step])
plt.legend(['%d' % i for i in range(10)], loc='best');